# 0-载入包

In [1]:
#注意添加注释

In [2]:
library(tidyverse)
library(readr)
library(readxl)
library(writexl)
library(lubridate)
library(data.table)
library(plm)
library(ggthemes)
library(ggpubr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'data.table'


The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:dplyr':

    between, first, last


The following object is masked from 'package:purrr':

    transpose



Attaching package: 'plm'


The following object is masked from 'package:data.table':

    between


The following objects are masked from 'pa

In [3]:
library(plm)

# 1-股票数据

## 1.1-生成文件名

In [4]:
#读入4个国泰安.csv。详细信息见 国泰安指标 和 金融结构memo。
#国泰安 文件名格式：D:/IDMhs/收益预测-月个股回报率文件-2024-09-23/SRFR_Amnthlyr.csv

In [5]:
# Read the Excel file
file_path <- "D:/jianguo/MyDoc/gta_name.xlsx"  #上述4个.csv的文件夹名称和文件名称在这个.xlsx中。最后一列是数据框名称
data <- read_excel(file_path)
#dir_name	file_name	data_name
#收益预测-月个股回报率文件	SRFR_Amnthlyr	srfrm

In [6]:
dir_base="D:/IDMhs"
dir_suffix="2024-09-23"    #suffix-后缀。prefix-前缀
dir_root = data$dir_name
file_root =  data$file_name

In [7]:
# Concatenate to generate complete file paths
#国泰安 文件名格式：D:/IDMhs/收益预测-月个股回报率文件-2024-09-23/SRFR_Amnthlyr.csv
#complete_file_paths <- paste0(dir_base, data$dir_name, "-", dir_suffix, "/", data$file_name, ".csv")
#file_name <- file.path(dir_base, paste0(dir_root, "-", dir_suffix), paste0(file_root, ".csv"))
complete_file_paths <- file.path(dir_base, paste0(dir_root, "-", dir_suffix), paste0(file_root, ".csv"))

In [8]:
# Print the results。因为打印函数 比较慢。详见 POE指南
print(complete_file_paths)
#print(file_name)

[1] "D:/IDMhs/收益预测-月个股回报率文件-2024-09-23/SRFR_Amnthlyr.csv"    
[2] "D:/IDMhs/股票市场-年个股回报率文件-2024-09-23/TRD_Year.csv"         
[3] "D:/IDMhs/股票市场-公司文件-2024-09-23/TRD_Co.csv"                   
[4] "D:/IDMhs/中国上市公司股权性质文件-2024-09-23/EN_EquityNatureAll.csv"


## 1.2-读入国泰安.csv文件

In [9]:

#注意：本来使用的是data.table的fread。但是最后一个.csv文件有些异常，最后一列的单元格，有些在末尾有个空格。
#fread无法处理前后的空格，会产生错误。
#readr的read_csv可以处理多于的空格
#show_col_types = FALSE。可以更加简洁地查看是否有错误信息。算了，还是展示更多一些信息。
safe_read_csv <- function(file_path) {
  tryCatch({
    read_csv(file_path)  #from package readr
  }, error = function(e) {
    warning(paste("Error reading file:", file_path, "\n", e$message))
    return(NULL)
  })
}

In [10]:
#df_names <- data$data_name   #在后面再赋值
#print(df_names)

In [11]:
# Read all CSV files
# 构建 空的向量 列表csv_list，存放 4个.csv。
# 后来结合 claude的提示。直接读入 数据框 给 变量  不需要 建立中间的 list 变量 
#算了，还是使用list 中间变量。因为下面进行了很多 错误测试。最后再把list元素 赋值给 数据框变量名 df_names
#初始化一个空列表。而这里使用vector(mode = "list", length = n)，函数形式确实稍有些不直白
csv_list <- vector("list", length(complete_file_paths))
names(csv_list) <- data$data_name

In [12]:
#这是简化版代码
# for (i in 1:length(file_name)) {
#   cat("Reading file", i, "of", length(file_name), ":", file_name[i], "\n")
#   assign(df_names[i], safe_read_csv(file_name[i]) ) 
# }

In [13]:
#读入4个.csv。存入向量列表csv_list  #相比1:length(file_name)，seq_along(file_name)更加稳健
for (i in seq_along(complete_file_paths)) {
  cat("Reading file", i, "of", length(complete_file_paths), ":", complete_file_paths[i], "\n")
  csv_list[[i]] <- safe_read_csv(complete_file_paths[i])
}

Reading file 1 of 4 : D:/IDMhs/收益预测-月个股回报率文件-2024-09-23/SRFR_Amnthlyr.csv 


Rows: 815680 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Stkcd, Trdmnt
dbl (6): Msmvttl, Dturn, Beta, Illiqd, Mretwd, Mretnd

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Reading file 2 of 4 : D:/IDMhs/股票市场-年个股回报率文件-2024-09-23/TRD_Year.csv 


Rows: 68388 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (3): Stkcd, Opndt, Clsdt
dbl  (13): Trdynt, Yopnprc, Yclsprc, Ynshrtrd, Ynvaltrd, Ysmvosd, Ysmvttl, N...
date  (1): Capchgdt

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Reading file 3 of 4 : D:/IDMhs/股票市场-公司文件-2024-09-23/TRD_Co.csv 


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 5755 Columns: 36
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (22): Stkcd, Stknme, Conme, Conme_en, Indcd, Indnme, Nindcd, Nindnme, N...
dbl   (9): Cuntrycd, PROVINCECODE, CITYCODE, Ipoprm, Ipoprc, Nshripo, Parval...
date  (5): Listdt, Estbdt, Favaldt, Ipodt, Statdt

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Reading file 4 of 4 : D:/IDMhs/中国上市公司股权性质文件-2024-09-23/EN_EquityNatureAll.csv 


Rows: 59096 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (11): Symbol, ShortName, RegisterAddress, OfficeAddress, LargestHolder,...
dbl   (3): LargestHolderRate, TopTenHoldersRate, Seperation
num   (2): ActualControllerNatureID, EquityNatureID
date  (1): EndDate

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [14]:
#查看读入时存在的异常状况。
#有几个公司的发行日期为0000-00-00，但是因为没有用到这个字段，使用的是上市日期。所以之前也没有发现这个问题
problems(csv_list[[3]])
#problems(trdc)


row,col,expected,actual,file
<int>,<int>,<chr>,<chr>,<chr>
598,29,date in ISO8601,0000-00-00,D:/IDMhs/股票市场-公司文件-2024-09-23/TRD_Co.csv
600,29,date in ISO8601,0000-00-00,D:/IDMhs/股票市场-公司文件-2024-09-23/TRD_Co.csv
1677,29,date in ISO8601,0000-00-00,D:/IDMhs/股票市场-公司文件-2024-09-23/TRD_Co.csv
4053,29,date in ISO8601,0000-00-00,D:/IDMhs/股票市场-公司文件-2024-09-23/TRD_Co.csv
4062,29,date in ISO8601,0000-00-00,D:/IDMhs/股票市场-公司文件-2024-09-23/TRD_Co.csv


In [15]:
# Remove NULL elements (failed reads) from the list
# 这个其实是稳健性报告。检查是否存在没有正确读入的.csv文件
csv_list <- Filter(Negate(is.null), csv_list)

In [16]:
# Print summary information  #后来认识到 此处使用cat 潜在提高了速度
# 似乎cat 和print 相似，第一次速度都不快。那就不修改了
#同时print 毕竟在正式运行中 不使用，只是在演示中使用。也不修改了
cat("\nTotal number of files attempted:", length(complete_file_paths), "\n")
cat("Number of files successfully read:", length(csv_list), "\n")


Total number of files attempted: 4 
Number of files successfully read: 4 


In [17]:
# Print information about the first few datasets
# 打印每个数据框的前面3行
#后来意识到，print(head(csv_list[[i]], 3)) 其实是在vscode中加快了运行速度。因为直接print tibble速度慢
#不是。因为 head(csv_list[[i]], 3) 也是一个tibble。所以上述函数还是在使用 print。
#注意：在循环里必须使用print，否则head本身不显示内容
cat("\nSummary of the first few datasets:\n")
for (i in seq_len(min(5, length(csv_list)))) {
  dataset_name <- names(csv_list)[i]
  cat("\nDataset", i, "(", dataset_name, "):\n")
  print(dim(csv_list[[i]]))
  print(head(csv_list[[i]], 3))
}


Summary of the first few datasets:

Dataset 1 ( srfrm ):
[1] 815680      8
# A tibble: 3 × 8
  Stkcd  Trdmnt   Msmvttl    Dturn  Beta    Illiqd Mretwd Mretnd
  <chr>  <chr>      <dbl>    <dbl> <dbl>     <dbl>  <dbl>  <dbl>
1 000001 1991-04 2118487. 0.000505    NA 0.000129  NA     NA    
2 000001 1991-05 1859497. 0.00709     NA 0.0000104 -0.122  0.236
3 000001 1991-06 1648521. 0.00115     NA 0.0000123 -0.113 -0.113

Dataset 2 ( trdy ):
[1] 68388    17
# A tibble: 3 × 17
  Stkcd  Trdynt Opndt Yopnprc Clsdt Yclsprc  Ynshrtrd   Ynvaltrd Ysmvosd Ysmvttl
  <chr>   <dbl> <chr>   <dbl> <chr>   <dbl>     <dbl>      <dbl>   <dbl>   <dbl>
1 000001   1991 04-03    49   12-31    29.4  37261400     9.00e8  1.32e6  2.63e6
2 000001   1992 01-02    29.4 12-31    44.5 178026000     6.93e9  3.00e6  5.99e6
3 000001   1993 01-03    44.5 12-31    22.4 224732897     8.32e9  3.24e6  6.02e6
# ℹ 7 more variables: Ndaytrd <dbl>, Yretwd <dbl>, Yretnd <dbl>,
#   Yarkettype <dbl>, Capchgdt <date>, Ahshrtrd_Y <dbl>

In [18]:
#这是存在异常的几行。对应下面的2,2 行。最后一个单元格末尾存在多余的空格
print(csv_list[[4]][25816:25818, ])

# A tibble: 3 × 17
  Symbol ShortName EndDate    RegisterAddress       OfficeAddress LargestHolder
  <chr>  <chr>     <date>     <chr>                 <chr>         <chr>        
1 300198 纳川股份  2023-12-31 福建省泉州市泉港区普… 福建省泉州市… 长江生态环保…
2 300199 翰宇药业  2011-12-31 广东省深圳市南山区高… 广东省深圳市… 曾少贵       
3 300199 翰宇药业  2012-12-31 广东省深圳市南山区高… 广东省深圳市… 曾少贵       
# ℹ 11 more variables: LargestHolderRate <dbl>, TopTenHoldersRate <dbl>,
#   ActualControllerName <chr>, ActualControllerNatureID <dbl>,
#   SharesNature <chr>, EquityNature <chr>, EquityNatureID <dbl>,
#   Hierarchy <chr>, Founder <chr>, Seperation <dbl>, Originator <chr>


In [19]:
print(csv_list[[4]][[25817, 17]])

[1] "曾少贵,曾少强,曾少彬三位自然人股东及外资法人股东 SAIF III Mauritius (China Investments) Limited,TQM Investment Limited,内资法人股东深圳市丰成投资有限公司,深圳市创新投资集团有限公司。"


In [20]:
#这也是一个有些异常的单元格。原本内容是："金叶现代之窗"
print(csv_list[[4]][6973, 5])

# A tibble: 1 × 1
  OfficeAddress                                  
  <chr>                                          
1 陕西省西安市南二环西段100号金叶现代之窗大厦五层


In [21]:
# Get the names of the data frames
#将 .xlsx中第3列的数据框名称提取，在下面赋值给各个数据框
df_names <- names(csv_list)

In [22]:
# Extract each data frame and assign it to a variable
for (i in seq_along(csv_list)) {
  assign(df_names[i], csv_list[[i]])
}

In [23]:
#测试一下
print(srfrm[1:2, 1:5])

# A tibble: 2 × 5
  Stkcd  Trdmnt   Msmvttl    Dturn  Beta
  <chr>  <chr>      <dbl>    <dbl> <dbl>
1 000001 1991-04 2118487. 0.000505    NA
2 000001 1991-05 1859497. 0.00709     NA


## 1.3-srfrm6-换手率

In [24]:
#直接让POE对原来的代码进行优化
#主要完成以下几件工作：选择股票代码、交易月份，换手率。将字符串Trdmnt转换为日期。
#读取年份。计算年度换手率=月换手率加总
#按照惯例，命名为srfrm6
srfrm6 <- srfrm %>%
  select(Stkcd, Trdmnt, Dturn) %>%
  mutate(
    Trdmnt = ym(Trdmnt),
    year = year(Trdmnt)    
  ) %>%
  group_by(Stkcd, year) %>%
  summarise(Dturn = sum(Dturn, na.rm = TRUE), .groups = "drop")

In [25]:
#第一次使用.groups = "drop"命令，检查是否取消了group属性
is.grouped_df(srfrm6)

[1] FALSE

In [26]:
#查看第1,2行内容
srfrm6[1:2,]

Stkcd,year,Dturn
<chr>,<int>,<dbl>
000001,1991,0.831919
000001,1992,2.687327


## 1.4-trdy3-交易额-市值

In [27]:
#Ynvaltrd交易额 Ysmvosd流通市值 Ysmvttl总市值 
#年度 交易额 流通市值 总体市值  后两者是 存量，应该是年末 前者是流量
#可能存在 一些问题，比如 有些股票只 交易 6个月。但是 交易额可能 也是 一个 度量（交易12个月的比6个月的流动性更好），所以还是使用年度加总
#单位调整为亿。交易额单位为元，市值为千元
trdy3 <- trdy %>%
  mutate(
    Stkcd,
    year = Trdynt,
    trdv = Ynvaltrd / 10^8,
    osv = Ysmvosd / 10^5,
    ttv = Ysmvttl / 10^5,
    turn=trdv/osv,
    .keep ="none"
  )

In [28]:
#查看第1,2行内容
trdy3[1:2,]

Stkcd,year,trdv,osv,ttv,turn
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
000001,1991,8.995448,13.20425,26.34211,0.6812541
000001,1992,69.280485,30.04553,59.94000,2.3058499


## 1.5-trdc2-公司信息

In [29]:
trdc2=trdc %>% select(Stkcd, Stknme, Nnindcd, Nnindnme, Sctcd, Markettype, PROVINCE, PROVINCECODE)

## 1.6-eqn4-层级

In [30]:
#股权性质中有混合的，选取第一个。同时层级中有叫国家的，有叫央企的。统一为央企
#因为所有股票里面，有记录的第一年是1990年，所以那些后来上市的公司，为了形成平衡面板，需要填充1991至year0年之间的行
#最初以为是1991年。后来使用 find_earliest_year_row 检测，其实是1990年 
#详细图表说明 参见 上市公司整理4 
eqn4 <- eqn %>%
  mutate(
  Stkcd = Symbol,
  year = year(EndDate),
  eqn = substr(EquityNature, 1, 1),
  eqnid = substr(EquityNatureID, 1, 1),
  hie = if_else(substr(Hierarchy, 1, 1) == "国", "央", substr(Hierarchy, 1, 1)),
  .keep = "none"
  ) %>%
  group_by(Stkcd) %>%
  mutate(
    year0 = min(year),
    freq = ifelse(year == year0, year0 - 1989, 1)
  ) %>%
  uncount(freq) %>%
  mutate(year = 1989 + row_number()) %>%
  ungroup() %>%
  select(Stkcd, year, eqn, eqnid, hie)

In [31]:
print(eqn4[1:5,])

# A tibble: 5 × 5
  Stkcd   year eqn   eqnid hie  
  <chr>  <dbl> <chr> <chr> <chr>
1 000001  1990 国    1     市   
2 000001  1991 国    1     市   
3 000001  1992 国    1     市   
4 000001  1993 国    1     市   
5 000001  1994 国    1     市   


## 1.6b-查看最早年份

In [32]:
#查看 最早年份 是 1990还是 1991
find_earliest_year_row <- function(df, year_column) {
  df %>%
    filter({{ year_column }} == min({{ year_column }}, na.rm = TRUE))
}

In [33]:
# Specify the exact names of the data frames you want to process
df_names <- c("srfrm6", "trdy3", "eqn4")

# Loop through each specified data frame
for (df_name in df_names) {
  cat("\nProcessing data frame:", df_name, "\n")
  
  # Get the current data frame
  df <- get(df_name)
  
  # Find the earliest year row(s)
  result <- find_earliest_year_row(df, year)
  
  # Print the earliest year
  cat("The earliest year is:", min(result$year, na.rm = TRUE), "\n\n")
  
  # Print details of the earliest entry/entries
  cat("Details of the earliest entry/entries:\n")
  print(result)
  
  cat("\n-----------------------------------\n")
}


Processing data frame: srfrm6 
The earliest year is: 1990 

Details of the earliest entry/entries:
# A tibble: 6 × 3
  Stkcd   year    Dturn
  <chr>  <int>    <dbl>
1 600601  1990 0.000485
2 600602  1990 0.00301 
3 600651  1990 0.0158  
4 600652  1990 0.0065  
5 600654  1990 0.00252 
6 600656  1990 0.000202

-----------------------------------

Processing data frame: trdy3 
The earliest year is: 1990 

Details of the earliest entry/entries:
# A tibble: 6 × 6
  Stkcd   year     trdv    osv     ttv     turn
  <chr>  <dbl>    <dbl>  <dbl>   <dbl>    <dbl>
1 600601  1990 0.000899 2.20    2.41   0.000409
2 600602  1990 0.00586  2.46   10.0    0.00239 
3 600651  1990 0.00202  0.132   0.132  0.0153  
4 600652  1990 0.000106 0.0165  0.0165 0.00639 
5 600654  1990 0.000200 0.0802  0.803  0.00249 
6 600656  1990 0.000245 1.36    7.46   0.000180

-----------------------------------

Processing data frame: eqn4 
The earliest year is: 1990 

Details of the earliest entry/entries:
# A tibble: 5,614

## 1.7-连接-join

In [34]:
#srfrm6$year = year(Trdmnt), Trdmnt是"1991-01"类型的字符串。trdy3$year=Trdynt是1991类型的双精度double。
#eqn4$year = year(EndDate) 是1991-12-31类型的date日期格式
#srfrm6$year是整型。trdy3$year 和 eqn4$year 是数值型 
str(srfrm6$year)
str(trdy3$year)
str(eqn4$year)

 int [1:73864] 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 ...
 num [1:68388] 1991 1992 1993 1994 1995 ...
 num [1:189189] 1990 1991 1992 1993 1994 ...


In [35]:
#将eqnid = NA 视为 其他 4。比如 京东方B  #由于eqn和eqnid是一一对应的，所以就不修改 eqn了
#新建一个字段 区分为 国有  非国有
mark5 <- trdy3 %>%
  left_join(srfrm6, by = c("Stkcd", "year")) %>%
  left_join(trdc2, by = "Stkcd") %>%
  left_join(eqn4, by = c("Stkcd", "year")) %>%
  mutate(
    eqnid = ifelse(is.na(eqnid), 4, eqnid),
    guo = as.integer(eqnid == 1)
  )

In [36]:
print(mark5[1:5,])

# A tibble: 5 × 18
  Stkcd   year  trdv   osv   ttv  turn Dturn Stknme   Nnindcd Nnindnme     Sctcd
  <chr>  <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <chr>    <chr>   <chr>        <dbl>
1 000001  1991  9.00  13.2  26.3 0.681 0.832 平安银行 J66     货币金融服务     2
2 000001  1992 69.3   30.0  59.9 2.31  2.69  平安银行 J66     货币金融服务     2
3 000001  1993 83.2   32.4  60.2 2.56  2.18  平安银行 J66     货币金融服务     2
4 000001  1994 97.1   31.2  45.2 3.12  2.74  平安银行 J66     货币金融服务     2
5 000001  1995 27.3   22.9  33.3 1.19  0.883 平安银行 J66     货币金融服务     2
# ℹ 7 more variables: Markettype <dbl>, PROVINCE <chr>, PROVINCECODE <dbl>,
#   eqn <chr>, eqnid <chr>, hie <chr>, guo <int>


In [37]:
earliest_row <- mark5 %>% filter(year == min(year, na.rm = TRUE))
print(earliest_row[, c(1:2, 8, 13:18)])

# A tibble: 6 × 9
  Stkcd   year Stknme   PROVINCE PROVINCECODE eqn   eqnid hie     guo
  <chr>  <dbl> <chr>    <chr>           <dbl> <chr> <chr> <chr> <int>
1 600601  1990 方正科技 上海市         310000 国    1     省        1
2 600602  1990 云赛智联 上海市         310000 国    1     省        1
3 600651  1990 飞乐音响 上海市         310000 国    1     省        1
4 600652  1990 退市游久 上海市         310000 民    2     NA        0
5 600654  1990 中安科   湖北省         420000 国    1     市        1
6 600656  1990 退市博元 广东省         440000 国    1     央        1


## 1.8-生成各省股票数据

In [38]:
#此处，POE的优化不如原始代码。使用原始代码
#为了防止和论文阐述中的turn混淆，将后两者重命名为 turn01, turn02
#备注三个换手率。Dturn是各个公司月度换手率加总。turn是各个公司的年度交易额/年末 流通市值。turn02=trdv/osv是省份层面计算
#按照是否国有 创建 新的变量guo
#pivot时 缺省变量名为 trdv_1  #奥卡姆剃刀。最后还是 变为 trdv1 变量
#最后依据地区PROVINCE只取两个字。只有 内蒙古 黑龙江 是 三个字。
#生成新变量region。放在原来第二位的year后面。第一位是 group_by 的 PROVINCE

mark6 = mark5 %>% group_by(PROVINCE,year) %>% summarise(trdv=sum(trdv), osv=sum(osv), ttv=sum(ttv), Dturn=mean(Dturn), turn01=mean(turn), turn02=trdv/osv ) 
#每个公司的平均mean  总体 交易额/流通市值
mark6b = mark5 %>% group_by(PROVINCE,year,guo) %>% summarise(trdv=sum(trdv), osv=sum(osv), ttv=sum(ttv), Dturn=mean(Dturn) , turn01=mean(turn), turn02=trdv/osv )
mark6c =  mark6b %>% pivot_wider(
    names_from = guo, 
    values_from = trdv:turn02, 
    names_glue = '{.value}{guo}'
)
mark7 = mark6 %>% left_join(mark6c)
mark7b= mark7 %>% ungroup()
mark8 = mark7b %>% mutate( region = str_sub(PROVINCE, 1, 2), .keep = "unused", .before = 1 )

`summarise()` has grouped output by 'PROVINCE'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'PROVINCE', 'year'. You can override using
the `.groups` argument.
Joining with `by = join_by(PROVINCE, year)`


In [39]:
print(mark6b[1:5,])

# A tibble: 5 × 9
# Groups:   PROVINCE, year [3]
  PROVINCE  year   guo      trdv     osv     ttv   Dturn  turn01  turn02
  <chr>    <dbl> <int>     <dbl>   <dbl>   <dbl>   <dbl>   <dbl>   <dbl>
1 上海市    1990     0  0.000106  0.0165  0.0165 0.0065  0.00639 0.00639
2 上海市    1990     1  0.00878   4.78   12.5    0.00642 0.00602 0.00184
3 上海市    1991     0  0.0162    0.334   0.334  0.366   0.0484  0.0484 
4 上海市    1991     1  6.51      6.15   19.8    1.82    0.996   1.06   
5 上海市    1992     0 41.2      12.0    57.1    1.86    4.29    3.44   


In [40]:
print(mark7b[1,])

# A tibble: 1 × 20
  PROVINCE  year    trdv   osv   ttv   Dturn  turn01  turn02    trdv0   trdv1
  <chr>    <dbl>   <dbl> <dbl> <dbl>   <dbl>   <dbl>   <dbl>    <dbl>   <dbl>
1 上海市    1990 0.00889  4.80  12.6 0.00644 0.00611 0.00185 0.000106 0.00878
# ℹ 10 more variables: osv0 <dbl>, osv1 <dbl>, ttv0 <dbl>, ttv1 <dbl>,
#   Dturn0 <dbl>, Dturn1 <dbl>, turn010 <dbl>, turn011 <dbl>, turn020 <dbl>,
#   turn021 <dbl>


In [41]:
print(mark8[1,])

# A tibble: 1 × 20
  region  year    trdv   osv   ttv   Dturn  turn01  turn02    trdv0   trdv1
  <chr>  <dbl>   <dbl> <dbl> <dbl>   <dbl>   <dbl>   <dbl>    <dbl>   <dbl>
1 上海    1990 0.00889  4.80  12.6 0.00644 0.00611 0.00185 0.000106 0.00878
# ℹ 10 more variables: osv0 <dbl>, osv1 <dbl>, ttv0 <dbl>, ttv1 <dbl>,
#   Dturn0 <dbl>, Dturn1 <dbl>, turn010 <dbl>, turn011 <dbl>, turn020 <dbl>,
#   turn021 <dbl>


# 2-宏观数据

## 2.0-说明

In [42]:
#进一步的具体信息。参见 金融结构code3 。
#备注：人口 城镇人口 全部时期 使用 CEIC。state 相对复杂单独处理。
#其他的数据 2005年之后 使用 CEIC（之前 新中国60年）。CEIC 虽然 准确一些，但是 前期 数据可能 不完整。
#d15=read_excel("d200-res-1b.xlsx") #新中国60年
#稳健性检验 对比 金融非线性-70-修正-2.xlsx 中 2005年北京 的数据 和 R 程序 中 的 数据。

In [43]:
#尝试 使用 CEIC - R 插件。算了。还是手动下载比较放心。在vscode中建立markdown，记录每个表格的搜索方法
#详见  CEIC指南   #第二次使用

## 2.1-读入CEIC.csv数据

In [44]:
#之前使用fread。但是 应该是gov trade gov 有 中文乱码（列名称中文乱码）。
#此次使用read_csv，可以比较稳健地处理乱码
#国泰安 文件名格式：D:/IDMhs/收益预测-月个股回报率文件-2024-09-23/SRFR_Amnthlyr.csv
#CEIC文件名格式： D:/jianguo/myR/pop-人口数-CEIC-2023-04-21.csv
#此脚本文件：D:/jianguo/myvscode/1-R/4-finstr.ipynb 
#此处 读入.csv文件，没有使用绝对路径，但是也成功了，因为最初将 D:/jianguo/myR/ 设为 工作目录。所以下面的代码中只是给出了文件名，就可以读入
#当然，最好还是使用 绝对路径

In [45]:
#数据信息文件：D:\jianguo\MyDoc\数据导入信息.xlsx
#第一列是id，后4列分别对应.csv文件名称。单位（原始单位 → 统一单位）。换算比率，数据框名称。比如：
#id	file_name	unit	uconv	data_name
#1	pop-人口数-CEIC-2023-04-21.csv	百万人 - 万	100	pop
#D:/jianguo/myR/pop-人口数-CEIC-2023-04-21.csv

In [46]:
dir_base2="D:/jianguo/myR/"

In [47]:
data_info=read_excel("D:/jianguo/MyDoc/数据导入信息.xlsx")
fname=data_info$file_name
uconv=data_info$uconv

In [48]:
cc1=1:13  #1-13是普通宏观数据 14-17是state数据（国有比重） 稍微特殊一些。需要 另外处理
fname1=fname[cc1] 
uconv1=uconv[cc1]

In [49]:
# print(fname1)
# print(uconv1)

In [50]:
#还是 read_csv彪悍。之前fread 读trade-进出口-中经-2023-04-23.csv时，存在乱码。还需要处理编码

In [51]:
name_zj=function(fna, ucon, ss, ee) {   
#zj-中经。fna-法纳-维拉
fnam = paste0(dir_base2, fna)  #使用绝对路径
pop_c=read_csv(fnam)  #fna: "pop-人口数-CEIC-2023-04-21.csv"  #最初是read_excel → fread → read_csv
#label=str_extract(fna, "^.*?(?=-)")  #label: "pop"  #变量名称  #用于后面变量赋值
label <- sub("-.*", "", fna)  #选择这个  #label= 数据框的名称 
#它将从第一个连字符"-"开始的所有内容替换为空字符串
a=names(pop_c)  #[1] "V1"       "人口数:北京"   "人口数:天津"
id=str_detect(a, "北京")  #获取 北京 所在的那一列的全部名称  #通常是第二列
nn=a[id]   #"人口数:北京" 
prefix <- sub("北京.*", "", nn)  #前缀 "人口数:"
id=nchar(prefix)  # 4
b=str_sub(a, start = id+1, end = id+2)  #省份名称 选取 2个字
b[1]="year"  #V1→year
names(pop_c)=b
#print(label)
#下面命令针对 信贷 其为 月度数据 格式为 "12/1987"
if (label %in% c("credit", "deposit" ) ) {
pop_d=pop_c %>% filter(str_sub(year, 1, 2) == "12")  
pop_c=pop_d %>% mutate( year = as.integer( str_sub(year, 4) ) , .before = 1)  #可以不用before 因为year本来就在第一位
}
#之所以叫 pop_c 是为了和上面的 非 信贷 数据一致
pop2= pop_c %>% filter(year>={{ss}} & year<={{ee}} )  #32个变量 year  北京  天津
#print(label)
pop3= pop2 %>% pivot_longer(
  cols = -1,
  names_to = "region",  #如果是字符串 则 为 引用 如果是变量 则代入值。此处第一句为字符串 第二句 为变量
  values_to = label   
)
#print(label)
pop4= pop3 %>% mutate(across( 3, ~ .* {{ucon}} )  )  #此处为 穿透
#print(label)
pop4
}

In [52]:
#从CEIC中选取2005年之后数据，之前的数据使用新中国60年
new_data=map2(fname1, uconv1, name_zj, ss=2005, ee=2021)  #fna, ucon  #13个变量  #new_data - 新数据

New names:
• `` -> `...1`
Rows: 74 Columns: 32
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (32): ...1, 人口数:北京, 人口数:天津, 人口数:河北, 人口数:山西, 人口数:内蒙古, 人口数:辽宁, 人口数:吉林, 人口数...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 51 Columns: 32
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (32): ...1, 北京:居民消费价格指数, 天津:居民消费价格指数, 河北:居民消费价格指数, 山西:居民消费价格指数, 内蒙古:居民消费...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 74 Columns: 32
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (32): ...1, 国内生产总值:北京:第一产业, 国内生产总值:天津:第一产业, 国内生产总值:河北:第一产业, 国内生产总值:山西:第一...

ℹ Use `spec()` to re

In [53]:
nd2=new_data %>% reduce(left_join)

Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`
Joining with `by = join_by(year, region)`


## 2.2-对数据进行初步整理

In [54]:
nd3=nd2 %>% mutate(mid = jun+sen , .keep ="unused" )

In [55]:
nd4=nd3 %>% mutate(cpi = cpi- 100 )

In [56]:
print(nd4[1,])

# A tibble: 1 × 14
   year region   pop   cpi first second   gdp  gdpi   gov   stu trade credit
  <dbl> <chr>  <dbl> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>  <dbl>
1  2005 北京    1538   1.5  86.9  1907. 7150.  112. 1058.  54.8 1255. 13834.
# ℹ 2 more variables: deposit <dbl>, mid <dbl>


In [57]:
#备注：咨询claude，使用print(nd4[1,], width = Inf)语法进行全部打印
#但是后来 发现 这种语法比较慢。所以 还是使用默认打印方法

In [58]:
#为了和 新中国60年对接。将列名按照字母顺序排序
#读入时13个变量。nd3中令初中生+高中生=中学生。所以现在为12个变量
nd5=nd4 %>% select(order(colnames(nd4) ) ) %>% relocate(year, region)

In [59]:
print(nd5[1,])

# A tibble: 1 × 14
   year region   cpi credit deposit first   gdp  gdpi   gov   mid   pop second
  <dbl> <chr>  <dbl>  <dbl>   <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>  <dbl>
1  2005 北京     1.5 13834.  26786.  86.9 7150.  112. 1058.  60.0  1538  1907.
# ℹ 2 more variables: stu <dbl>, trade <dbl>


## 2.2-读入新中国60年

In [60]:
d15=read_excel("d200-res-1b.xlsx")  #包含id  #北京=1 天津=2  #金融非线性-d200
#因为 这是 从 stata中导出的，而stata 必须数值型的变量才能xtset

d16=d15 %>% select(year:mid)  #2+14 = 16个变量
d17= d16 %>% mutate(region =str_sub(region,1,2))  #只选取 前面两个字 
d17b=d17 %>% select(-c(inv,state) )  #2017年以后 不报告 固定资本形成 和 固定资产投资 的数据
#state 最后直接 并入

d17c=d17b %>% select(order(colnames(d17b) ) ) %>% relocate(year, region)  #全部排序  #12个变量 + year, region

d17d= d17c %>% filter(year<2005)
d17e=bind_rows(d17d, nd5)  #合并 新中国60年和CEIC数据

# 3-国有比重

In [61]:
#最初是从1986年开始，后来为了调整1998年的数据，乃从1998年开始使用CEIC数据。
#1978-1997仍然用新中国60年
#1998年后的计算。1998-2011年使用总产值比例。2012年以后，使用营业收入的比例，应该是总产值不报告了

In [62]:
#结合claude优化
# Process state data (14-15)
state_data <- map2(fname[14:15], uconv[14:15], name_zj, ss = 1998, ee = 2011)

state <- state_data %>% 
  reduce(left_join) %>%
  mutate(state = nat / all) %>%
  select(year, region, state)

# Process state income data (16-17)
state_inc_data <- map2(fname[16:17], uconv[16:17], name_zj, ss = 2012, ee = 2021)

state_inc <- state_inc_data %>% 
  reduce(left_join) %>%
  mutate(state = nat / all) %>%
  select(year, region, state)

# Combine all state data
state80 <- bind_rows(
  d17 %>% filter(year < 1998) %>% select(year, region, state),
  state,
  state_inc
)
#将三个时期的state连接在一起 略等于80年

New names:
• `` -> `...1`
Rows: 34 Columns: 32
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (32): ...1, 工业总产值:北京, 工业总产值:天津, 工业总产值:河北, 工业总产值:山西, 工业总产值:内蒙古, 工业总产值:辽宁,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 29 Columns: 32
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (32): ...1, 工业总产值:国有及国有控股企业:北京, 工业总产值:国有及国有控股企业:天津, 工业总产值:国有及国有控股企业:河北, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Joining with `by = join_by(year, region)`
New names:
• `` -> `...1`
Rows: 28 Columns: 32
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (32): ...1, 营业收入:工业企业:北京, 营业收入:工业企业:天津, 营业收入:工业企业:河北, 营

# 4-进一步合并-股票

In [63]:
#如上所示，由于有些数据是分段拼接的。所以可能出现 1978-1985- 北京 然后 1978-1985 - 天津 的情形。为了谨慎起见，对每个地区 按照 时间排序
#1990年开始 的 总体数据。mark8对应股票数据

In [64]:
# Read region data
regn <- read_excel("D:/jianguo/MyDoc/region-常用.xlsx")

# Join data and process
d20 <- d17e %>%
  left_join(state80) %>%
  left_join(regn) %>%
  group_by(id) %>%
  arrange(year, .by_group = TRUE) %>%
  filter(year >= 1990) %>%
  left_join(mark8) %>%
  mutate(across(everything(), ~replace(., is.na(.), 10^-9))) %>%
  ungroup() %>%
  select(-id)

Joining with `by = join_by(year, region)`
Joining with `by = join_by(region)`
Joining with `by = join_by(year, region)`


In [65]:
print(d20[1,])

# A tibble: 1 × 33
   year region   cpi credit deposit first   gdp  gdpi   gov   mid   pop second
  <dbl> <chr>  <dbl>  <dbl>   <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>  <dbl>
1  1990 北京     5.4   573.    894.  43.9  501.  105.  66.5  40.9  1086   262.
# ℹ 21 more variables: stu <dbl>, trade <dbl>, state <dbl>, trdv <dbl>,
#   osv <dbl>, ttv <dbl>, Dturn <dbl>, turn01 <dbl>, turn02 <dbl>, trdv0 <dbl>,
#   trdv1 <dbl>, osv0 <dbl>, osv1 <dbl>, ttv0 <dbl>, ttv1 <dbl>, Dturn0 <dbl>,
#   Dturn1 <dbl>, turn010 <dbl>, turn011 <dbl>, turn020 <dbl>, turn021 <dbl>


# 5-计算相对比重

In [66]:
#cpi, credit, deposit, first, gdp, gdpi, gov, mid, pop, second, state, stu, trade  #13个变量 舍弃了 inv 因为没有后续数据

In [67]:
m3d <- d20 %>%
  # Calculate ratios for credit, deposit, first, second, gov, trade
  mutate(across(c("credit", "deposit", "first", "second", "gov", "trade"),
                list(a = ~ . / gdp),
                .names = "{col}{fn}")) %>%
  # Calculate ratios for stu and mid
  mutate(across(c("stu", "mid"),
                list(a = ~ . / pop),
                .names = "{col}{fn}"))

In [68]:
#调整北京的 tradea
#北京比率太大，使用天津估计
#bb=mean(b$rate)   #[1] 1.760094
#暂时按照原来的数据，用1.8进行调整  #具体代码参见 金融结构9

In [69]:
a4= m3d %>%
    select(year, region, tradea) %>%
      pivot_wider(names_from = region, values_from = tradea) %>%
      mutate(北京 = if_else(year %in% 1978:1995, 天津 * 1.8, 北京)) %>%
      pivot_longer(cols = -year, names_to = "region", values_to = "tradea")

In [70]:
print(a4[1,])

# A tibble: 1 × 3
   year region tradea
  <dbl> <chr>   <dbl>
1  1990 北京    0.128


In [71]:
m4= m3d %>% select(-tradea) %>% left_join(a4)

Joining with `by = join_by(year, region)`


In [72]:
print(m4[1,], width = Inf)

# A tibble: 1 × 41
   year region   cpi credit deposit first   gdp  gdpi   gov   mid   pop second
  <dbl> <chr>  <dbl>  <dbl>   <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>  <dbl>
1  1990 北京     5.4   573.    894.  43.9  501.  105.  66.5  40.9  1086   262.
    stu trade state        trdv         osv         ttv       Dturn      turn01
  <dbl> <dbl> <dbl>       <dbl>       <dbl>       <dbl>       <dbl>       <dbl>
1  14.0  236. 0.717 0.000000001 0.000000001 0.000000001 0.000000001 0.000000001
       turn02       trdv0       trdv1        osv0        osv1        ttv0
        <dbl>       <dbl>       <dbl>       <dbl>       <dbl>       <dbl>
1 0.000000001 0.000000001 0.000000001 0.000000001 0.000000001 0.000000001
         ttv1      Dturn0      Dturn1     turn010     turn011     turn020
        <dbl>       <dbl>       <dbl>       <dbl>       <dbl>       <dbl>
1 0.000000001 0.000000001 0.000000001 0.000000001 0.000000001 0.000000001
      turn021 credita deposita firsta seconda  gova   stua   m

## 5.1-计算实际gpd

In [73]:
byear=1990   #byear - base year
m4b= m4 %>% mutate( gdpx=ifelse(year==byear, gdp, gdpi/100)  )  #gdpx - gdpc的中间阶段
#基年（1978）设置为当年的名义GDP，后面设置为实际GDP指数（以小数表示，而非百分数）
m5= m4b %>% group_by(region) %>% mutate( gdpc=cumprod(gdpx))  #累积乘积  #这个命令适用于 第一行为base year
#前面版本【R代码复盘】的命令 可以 扩展到 任意年 为 base year
#c表示不变价格constant  #引入了group

m5b= m5 %>% mutate(pgdpc=gdpc/pop, lny=log(pgdpc) )
#此处 保持 group="region" 后面 私营信贷 可以使用


In [76]:
str(m5b)

gropd_df [992 × 45] (S3: grouped_df/tbl_df/tbl/data.frame)
 $ year    : num [1:992] 1990 1991 1992 1993 1994 ...
 $ region  : chr [1:992] "北京" "北京" "北京" "北京" ...
 $ cpi     : num [1:992] 5.4 11.9 9.9 19 24.9 17.3 11.6 5.3 2.4 0.6 ...
 $ credit  : num [1:992] 573 660 770 972 1238 ...
 $ deposit : num [1:992] 894 1134 1364 1649 2279 ...
 $ first   : num [1:992] 43.9 45.8 49.1 53.7 67.5 ...
 $ gdp     : num [1:992] 501 599 709 886 1145 ...
 $ gdpi    : num [1:992] 105 110 111 112 114 ...
 $ gov     : num [1:992] 66.5 68 71.7 81 98.5 ...
 $ mid     : num [1:992] 40.9 43.2 47.7 53.2 58.4 ...
 $ pop     : num [1:992] 1086 1094 1102 1112 1125 ...
 $ second  : num [1:992] 262 292 346 420 518 ...
 $ stu     : num [1:992] 14 13.7 14 15.9 17.5 ...
 $ trade   : num [1:992] 236 242 250 279 289 ...
 $ state   : num [1:992] 0.717 0.696 0.678 0.651 0.553 ...
 $ trdv    : num [1:992] 1.00e-09 1.00e-09 4.97 3.49e+01 2.54e+02 ...
 $ osv     : num [1:992] 1.00e-09 1.00e-09 2.03 7.02 1.75e+01 ...
 $ ttv   

# 6-计算私营信贷

In [74]:
d80m=pdata.frame(m5b, index = c("region", "year"))
d_lm=pvcm(credita~state, data=d80m,effect = "time",model ="within")
two=coef(d_lm)

a=two[,1]  #a是民企
d=two[,2]  #d是差异
beta=a+d  #beta是 国企。 计算公式参见稿纸1074  #2023-04-10稿纸 226  或者  0金融抑制-实证2.doc

#如下 统一命名 以后缀a表示比例。同时用0表示初步的估计，而正式估计则不加后缀
#state 国有比重 stat国有信贷额 stata 国有信贷比重 priv 私营信贷额 priva 私营信贷比例
#dep=staty+privy=beta*state + a*(1-state) =a + (beta-a) * state = a+ d *state
m5d = m5b %>% mutate(stata0=beta*state, priva0=a*(1-state), 
    relat=priva0/stata0,props=stata0/(stata0+priva0),
    stata=credita*props, priva=credita*(1-props))
#beta=38，state=38*n。所以扩展相乘。但是此处之所以有意义，  #t=38或 其他 年数

#上述是期望值。但是实际值和期望值有偏差。
#此处是 方法2 保证了staty2+privy2=dep
#前面 保持 group="region" 此处可以使用 
m6=m5d %>% mutate(across (c("lny","credit","deposit"),
    .fns = list(d = ~ . - dplyr::lag(.)   ) , 
    .names = "{fn}{col}" ) )
#此处将d放在前面  #计算增量
m6b = m6 %>% mutate(across (c("dcredit","ddeposit"),
    .fns = list( a=~./gdp), 
    .names = "{col}{fn}" ) )
#此处比例是百分数

# 7-继续计算

## 7.1-金融结构